In [124]:
using LinearAlgebra
using ForwardDiff

In [125]:
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end

hat (generic function with 1 method)

In [126]:
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end

L (generic function with 1 method)

In [127]:
function R(q)
    s = q[1]
    v = q[2:4]
    R = [s    -v';
         v  s*I-hat(v)]
    return R
end

R (generic function with 1 method)

In [129]:
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I]

4×3 Array{Float64,2}:
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [130]:
function G(q)
    G = L(q)*H
end

G (generic function with 1 method)

In [132]:
#Generate a random quaternion
qtrue = randn(4)
qtrue = qtrue/norm(qtrue)

Qtrue = H'*(R(qtrue)'*L(qtrue))*H #Generate equivalent rotation matrix

3×3 Array{Float64,2}:
 -0.117809  -0.95803    0.261343
  0.809805   0.0596364  0.58366
 -0.574749   0.280398   0.768792

In [133]:
#Generate data

vN = randn(3,10) #Generate some random inertial vectors

#normalize
for k = 1:10
    vN[:,k] .= vN[:,k]./norm(vN[:,k])
end

vB = Qtrue'*vN #generate body-frame vectors

3×10 Array{Float64,2}:
 -0.923321  0.101344   0.670826  …  -0.760282  -0.895379   -0.626618
 -0.116872  0.519615  -0.72543      -0.189918  -0.0701578   0.597101
  0.365813  0.848369  -0.154088     -0.621211  -0.439742   -0.50082

In [134]:
function residual(q)
    r = vN - ((R(q)'*L(q))[2:4,2:4])*vB
    return r[:]
end

residual (generic function with 1 method)

In [163]:
#Random initial guess
q = randn(4)
q = q/norm(q)

4-element Array{Float64,1}:
 -0.06602767505435467
  0.024102413816553554
 -0.9385481942302594
 -0.3379152362386927

In [164]:
#Gauss-Newton Method
ϕ = ones(3)
iter = 0
while maximum(abs.(ϕ)) > 1e-8
    r = residual(q)
    dr = ForwardDiff.jacobian(residual, q)
    ∇r = dr*G(q)
    ϕ = -(∇r'*∇r)\(∇r'*r) #3-parameter update computed with gauss-newton
    q = L(q)*[sqrt(1-ϕ'*ϕ); ϕ] #multiplicative update applied to q
    iter += 1
end

In [165]:
iter

6

In [166]:
q-qtrue

4-element Array{Float64,1}:
 -1.1102230246251565e-16
 -2.7755575615628914e-17
  0.0
  0.0

In [167]:
q+qtrue

4-element Array{Float64,1}:
 -1.3079063673690854
  0.23186851112915566
 -0.6392602817942956
 -1.35165240349079

In [123]:
T

4×4 Diagonal{Float64,Array{Float64,1}}:
 1.0    ⋅     ⋅     ⋅ 
  ⋅   -1.0    ⋅     ⋅ 
  ⋅     ⋅   -1.0    ⋅ 
  ⋅     ⋅     ⋅   -1.0